In [1]:
from math import sqrt, pi, cos, sin, radians, atan2, asin
import random
import numpy as np
import pickle
import gurobipy as gp
from gurobipy import GRB

In [2]:
def pos2dist(point1,point2):
    radius = 3958.756;
    lat1 = point1[0]*pi/180
    lat2 = point2[0]*pi/180
    lon1 = point1[1]*pi/180
    lon2 = point2[1]*pi/180
    deltaLat = lat2-lat1;
    deltaLon = lon2-lon1;
    a = sin((deltaLat)/2)**2 + cos(lat1)*cos(lat2) * sin(deltaLon/2)**2
    c = 2*atan2(sqrt(a),sqrt(1-a))
    x = deltaLon*cos((lat1+lat2)/2)
    y = deltaLat
    d1 = radius*c
    d2 = radius*sqrt(x*x + y*y)
    return d1


def intilalize_tours(ins):
    with open("HSCPP"+str(data_number)+"_ins"+ str(ins) + "tours", "rb") as fp:   # Upickling
        tours = pickle.load(fp)
    with open("HSCPP"+str(data_number)+"_ins"+ str(ins) + "prize", "rb") as fp:   # Unpickling
        prize_cd_vec = pickle.load(fp)
    with open("HSCPP"+str(data_number)+"_ins"+ str(ins) + "IT", "rb") as fp:   # Unpickling
        IT_vec = pickle.load(fp)
        
            
    masterip = gp.Model("iP") # master LP problem
    masterip.ModelSense=-1   # maximize
    masterip.setParam('OutputFlag', 0) 
    z_ip = {}
    T = len(tours)
    for t in range(T):
        z_ip[t] = masterip.addVar(obj=prize_cd_vec[t], vtype="B")
    nodes={}
    T = len(tours)
    a = [[0] * 80 for _ in range(T)]
    for k in range(T):
        for i in tours[k]:
            a[k][i] = 1
    for i in range(80):
        coef = [a[k][i] for k in range(T) if a[k][i] > 0]
        var = [z_ip[k] for k in range(T) if a[k][i] > 0]
        nodes[i] = masterip.addLConstr(gp.LinExpr(coef,var), "<=", 1)
    masterip.addConstr(sum(-IT_vec[k] * z_ip[k] for k in range(T)) <= -4)
    masterip.addConstr(sum(z_ip[k] for k in range(T))== 3)
    masterip.optimize()
    sol_tour = []
    prize_cd = 0
    for t in range(T):
        if z_ip[t].X > 0.5:
            sol_tour.append(t)
            prize_cd += prize_cd_vec[t]
    print(sol_tour)
    intial_tours = [tours[sol_tour[0]]] +  [tours[sol_tour[1]]] +  [tours[sol_tour[2]]]
    # for i in range(3):
    #     dist = 0
    #     IT = 0
    #     for j in range(len(intial_tours[i])-1):
    #         dist += dist_80[intial_tours[i][j]][intial_tours[i][j+1]]
    #         if intial_tours[i][j] in inner and intial_tours[i][j+1] in inner:
    #             IT += dist_80[intial_tours[i][j]][intial_tours[i][j+1]]
    #     if intial_tours[i][-1] in inner and intial_tours[i][0] in inner:
    #         IT += dist_80[intial_tours[i][-1]][intial_tours[i][0]]
    #     dist += dist_80[intial_tours[i][-1]][intial_tours[i][0]]
    #     IT += (tau-dist)
    #     print(dist, IT)
    for i in range(len(intial_tours)):
        for j in range(len(intial_tours[i])):
            if intial_tours[i][j] in inner:
                intial_tours[i][j] = inner_virtual_all[inner.index(intial_tours[i][j])]
                break
    return intial_tours

In [3]:
def ip_solver_st(Prize_set, inner_set, inner_real_set, outer_set, dist_set,intial_tours):
    sPrize = sum(Prize_set) - len(inner_real_set)
    N = len(Prize_set)
    print(N)
    v = {}
    cnt = N - len(inner_real_set)
    for i in inner_real_set:
        v[i] = cnt
        cnt += 1
    m = gp.Model("m")
    m.setParam('OutputFlag', 1)  # change to 1 to show Gurobi solver output
    m.setParam('TIME_LIMIT', 4*3600)
    m.setParam('LogFile', 'test1.log')
    m.setParam('DisplayInterval', 60)
#     m.setParam('MIPGap', 0.05)
    x = m.addVars(N, N, K, vtype=GRB.BINARY, name="x")
    y = m.addVars(N, K, vtype=GRB.BINARY, name="y")
    m.setObjective(sum(Prize_set[i] * y[i, k] for i in range(N) for k in range(K)),
                   GRB.MAXIMIZE)
    m.addConstrs(x.sum(i, '*', k)  == y[i, k] for i in range(N) for k in range(K))
    m.addConstrs(y.sum(i, '*') <= 1 for i in range(N))
    m.addConstrs(x.sum(i, '*', k) - x.sum('*', i, k) == 0 for i in range(N) for k in range(K))
    m.addConstrs(sum(x[i, j, k] for i in inner_set for j in outer_set) == 1 for k in range(K))
    m.addConstrs(sum(x[i, j, k] for i in outer_set for j in inner_set) == 1 for k in range(K))
    m.addConstrs(sum(y[i, k] for i in range(nCusts, N)) == 1 for k in range(K))
    m.addConstrs(sum(y[i, k] + y[v[i], k] for k in range(K)) <= 1 for i in inner_real_set)
    m.addConstrs(sum(dist_set[i, j] * x[i, j, k] for i in range(N) for j in range(N)) <= tau for k in range(K))
    m.addConstr(sum(dist_set[i, j] * x[i, j, k] for i in inner_set for j in inner_set for k in range(K)) + (tau * K
                - sum(dist_set[i, j] * x[i, j, k] for i in range(N) for j in range(N) for k in range(K))) >= tau)
    m.addConstrs(x[i, i, k] == 0 for i in range(N) for k in range(K))
    for k in range(3):
        for i in range(len(intial_tours[k])-1):
            x[intial_tours[k][i], intial_tours[k][i+1], k].start = 1
            y[intial_tours[k][i+1], k].start  = 1
        x[intial_tours[k][-1], intial_tours[k][0], k].start = 1
        y[intial_tours[k][0], k].start = 1
    m.update()
    m._varsx = x
    m._varsy = y
    m.Params.lazyConstraints = 1
    m.optimize(subtourelim)
    vals = m.getAttr('x', x)
    sol = {}
    prize_cd_vec = [0] * 3
    L_vec = [0] * 3
    it_inner = [0] * 3
    it_inner_waiting = [0] * 3
    for k in range(K):
        selected = []
        ls = []
        for i, j, l in vals.keys():
            if vals[i, j, k] > 0.5 and k == l:
                selected.append((i, j))
                ls.append(i)
        selected = gp.tuplelist(selected)
        sol[k] = subtour(selected, ls)[0]
        dist = 0
        for i in range(len(sol[k])):
            prize_cd_vec[k] += Prize_set[sol[k][i]]
        for i in range(len(sol[k])-1):
            dist += dist_set[sol[k][i], sol[k][i+1]]
            if sol[k][i] in inner_set and sol[k][i+1] in inner_set:
                it_inner[k] += dist_set[sol[k][i], sol[k][i+1]]
        if sol[k][0] in inner_set and sol[k][-1] in inner_set:
            it_inner[k] += dist_set[sol[k][-1]][sol[k][0]]
        dist += dist_set[sol[k][-1]][sol[k][0]]
        L_vec[k] = dist
        it_inner[k] /= 5.2
        if dist <= tau:
            it_inner[k] += (tau-dist)/5.2
            it_inner_waiting[k] = (tau - dist)/5.2
    return m.objVal, m.objBound, L_vec, prize_cd_vec, it_inner
#     return sol, prize_cd, it_inner, it_inner_waiting
        



def subtourelim(model, where):
    if where == GRB.Callback.MIPNODE:
        depth = model.cbGet(GRB.Callback.MIPNODE_NODCNT)
        if depth == 0:
            if model.cbGet(GRB.Callback.MIPNODE_STATUS) == GRB.Status.OPTIMAL:
                valx = model.cbGetNodeRel(model._varsx)
                valy = model.cbGetNodeRel(model._varsy)
                M = nCusts
                for k in range(K):
                    m1 = gp.Model("m1")
                    m1.setParam('OutputFlag', 0)  # change to 1 to show Gurobi solver output
                    w = m1.addVars(M, M, vtype=GRB.BINARY, name="w")
                    z = m1.addVars(M, vtype=GRB.BINARY, name="z")
                    zz = m1.addVars(M, vtype=GRB.BINARY, name="zz")
                    m1.setObjective(sum(valx[i, j, k] * w[i, j] for i in range(M) for j in range(M)) - sum(valy[i, k] * z[i]
                                                                                                        for i in
                                                                                                        range(M)) + sum(
                        valy[i, k] * zz[i] for i in range(M)), GRB.MAXIMIZE)
                    m1.addConstrs(w[i, j] <= z[i] for i in range(M) for j in range(M))
                    m1.addConstrs(w[i, j] <= z[j] for i in range(M) for j in range(M))
                    m1.addConstrs(zz[i] <= z[i] for i in range(M))
                    m1.addConstr(sum(zz[i] for i in range(M)) == 1)
                    m1.optimize()
                    # print(m1.objVal)
                    if m1.objVal > 0:
                        sol = []
                        lhs = gp.LinExpr()
                        rhs = gp.LinExpr()
                        for i in range(M):
                            if z[i].X > 0.5:
                                sol.append(i)
                            if zz[i].X > 0.5:
                                speical = i
                        # print(sol)
                        for i in sol:
                            if i != speical:
                                rhs.add(model._varsy[i, k])
                            for j in sol:
                                lhs.add(model._varsx[i, j, k])
                        model.cbCut(lhs <= rhs)
    if where == GRB.Callback.MIPSOL:
        # make a list of edges selected in the solution
        valx = model.cbGetSolution(model._varsx)
        for k in range(K):
            ls = []
            selected = []
            for i,j,l in model._varsx.keys():
                if l == k:
                    if valx[i,j,k] > 0.5:
                        ls.append(i)
                        selected.append((i, j))
            selected = gp.tuplelist(selected)
        # find the shortest cycle in the selected edge list
            tour = subtour(selected, ls)
        #print('inside subtour elim')
        # print('In exact solve, eliminated following subtour:')
        #print(n)
            for asubtour in tour:
                if max(asubtour) < nCusts:
                    lhs = gp.LinExpr()
                    for i in asubtour:
                        for j in asubtour:
                            lhs.add(model._varsx[i, j, k])
                    for i in asubtour:
                        rhs = gp.LinExpr()
                        for j in asubtour:
                            if i != j:
                                rhs.add(model._varsy[j, k])
                        model.cbLazy(lhs <= rhs)


# Given a tuplelist of edges, find the shortest subtour

def subtour(edges, ls):
    unvisited = ls
    cycle = []# initial length has 1 more city
    while unvisited: # true if list is non-empty
        thiscycle = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            unvisited.remove(current)
            neighbors = [j for i,j in edges.select(current,'*') if j in unvisited]
        cycle.append(thiscycle)
    return cycle

In [4]:
data_number = 1
K = 3
tau = 26 * 0.8
with open("HSCPP_real_world_" + str(data_number), "rb") as fp:   # Unpickling
    [Prize_80_vec, dist_80, marker_locations] = pickle.load(fp)

nCusts = 80
x0, y0 = 39.16021, -77.20217

In [5]:
inner_real_all = []
inner_virtual_all = []
outer_all = []
cnt = nCusts
ix = {}
for i in range(nCusts):
    ix[i] = i
    if pos2dist((x0,y0), marker_locations[i]) <= 1.4:
        inner_real_all.append(i)
        inner_virtual_all.append(cnt)
        ix[cnt] = i     
        cnt += 1
        marker_locations.append(marker_locations[i])
    else:
        outer_all.append(i)
inner = inner_real_all + inner_virtual_all
N = len(marker_locations)
dist_aug = np.zeros((N, N))
for i in range(N):
    for j in range(N):
        if i != j:
            dist_aug[i][j] = dist_80[ix[i]][ix[j]]  

In [ ]:
prize_cd_ip1 = [0] * 5
gap_ip1  = [0] * 5
ub = [0] * 5
L_val = []
Prize_val = []
IT_val = []
for ins in range(1):
    Prize_80 = Prize_80_vec[ins]
    Prize_80 += [1] * len(inner_real_all)
    intial_tours = intilalize_tours(ins)
    prize_cd, gap, L_vec, prize_cd_vec, it_inner = ip_solver_st(Prize_80, inner, inner_real_all, outer_all, dist_aug, intial_tours)
    L_val += L_vec
    Prize_val += prize_cd_vec
    IT_val += it_inner
    prize_cd_ip1[ins] = prize_cd
    ub[ins] = gap
    gap_ip1[ins] = (gap-prize_cd)/gap*100     

Academic license - for non-commercial use only - expires 2023-07-03
Using license file /Users/clearlove/gurobi.lic
[0, 1, 2]
108
Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Changed value of parameter TIME_LIMIT to 14400.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Changed value of parameter LogFile to test1.log
   Prev:   Default: 
Changed value of parameter DisplayInterval to 60
   Prev: 5  Min: 1  Max: 2000000000  Default: 5
Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1121 rows, 35316 columns and 182952 nonzeros
Model fingerprint: 0x80b6fe38
Variable types: 0 continuous, 35316 integer (35316 binary)
Coefficient statistics:
  Matrix range     [1e-01, 5e+02]
  Objective range  [1e-02, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+01]

Use